# Physics 300 
## Computational Physics I (Fall 2018)
## BPB-248, Mon/Wed 02:30 - 03:45 pm 

|Instructor| Prof. Qiang Zhu|
|--|-------------------------------|
|Email | qiang.zhu@unlv.edu|
|Website|http://www.physics.unlv.edu/~qzhu/|
|Office| BPB 232|
|Office hours | Mon/Wed 03:45 - 05:00 pm |

# 20 Global Optmization (III)

## 20.1 Genetic Algorithm

A genetic algorithm is a search heuristic that is inspired by Charles Darwin’s theory of natural evolution. This algorithm reflects the process of natural selection where the fittest individuals are selected for reproduction in order to produce offspring of the next generation.

- 1, Initialization
- 2, Selection
- 3, Genetic operators (crossover/mutation)
- 4, Termination


## 20.2 Application to LJ clusters


### Cambridge cluster database

Global optimization of LJ clusters has been one of the most interesting subject in computational physics/chemistry community. It has become a gold standard test bed when people wants to propose a new method.

Below is link to those reported geometry and energy values collected by Wales's group in Cambridge.

- [N = 1-150](http://doye.chem.ox.ac.uk/jon/structures/LJ/tables.150.html)
- [N = 310-561](http://doye.chem.ox.ac.uk/jon/structures/LJ/LJ310-561.html) 

In [1]:
from scipy.optimize import minimize
import numpy as np
from random import sample
from numba import jit
from time import time

@jit
def LJ(r):
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

@jit
def total_energy(points): #points is a 3*N array of all the atoms involved
    total = 0
    num_atoms = int(len(points)/3)
    
    for i in range(num_atoms - 1):
        for j in range(i+1, num_atoms):
            pos1 = points[i*3 : (i+1)*3]
            pos2 = points[j*3 : (j+1)*3]
            
            r = np.linalg.norm(pos1-pos2)
            
            total += LJ(r)
    return total

@jit
def gradient(pos):
    """
    Calculate and return the Jacobian for minimizing
    the Lennard-Jones potential between N atoms.
    Input:
        positions: 3*N 1-D array of x,y,z positions of each atom
    Output:
        3*N 1-D array of the Jacobian elements for N atoms.
    """
    N = int(len(pos)/3)
    J = np.zeros([N, 3]) 
    
    for i in range(N-1):
        center = pos[3*i:(3*i)+3]
        for j in range(i+1, N):
            neighbor = pos[3*j:(3*j)+3] 
            r = np.linalg.norm(center - neighbor)
            J[i] += (24/r**7 - 48/r**13)*(center - neighbor)/r
    
    return J.flatten()
            
def init_pos(N, L=5):
    return L*np.random.random_sample((N*3,))

def init_population(population_size, N_atom):
    cluster = []
    for i in range(population_size):
        cluster.append(init_pos(N_atom))
    return np.array(cluster)

def local_optimize(cluster):
    fitness = []
    for i, cluster0 in enumerate(cluster):
        res = minimize(total_energy, cluster0, method='Powell', tol=1e-4, options={'maxiter':20}) #opt w/o gradient
        res = minimize(total_energy, res.x, method='CG', jac=gradient, tol=1e-4) #analytical gradient
        #res = minimize(total_energy, cluster0, method='CG', tol=1e-3) 
        cluster0 = res.x
        fitness.append(res.fun)
    return cluster, np.array(fitness)

def selTournament(fitness, factor=0.35):
    """
    Select the best individual among *tournsize* randomly chosen
    individuals, *k* times. The list returned contains
    references to the input *individuals*.
    """
    IDs = sample(set(range(len(fitness))), int(len(fitness)*factor))
    min_fit = np.argmin(fitness[IDs])
    return IDs[min_fit]

def ranking(fitness):
    return np.argsort(fitness)

def mutation(cluster, fitness, kT=0.5):
    """
    Select a structure and perturb it
    """
    id = selTournament(fitness)
    cluster0 = cluster[id]
    disp = np.random.random_sample((len(cluster0),))-0.5
                                   
    return cluster0 + kT*disp

def crossover(cluster, fitness):
    """
    Select two structures and do sort of *crossover*
    """
    id1 = selTournament(fitness)
    while True:
        id2 = selTournament(fitness)
        if id2 != id1:
            break
    frac = np.random.random()
    #todo: center the cluster, and then xA+(1-x)B 
    return cluster[id1]*frac + cluster[id2]*(1-frac)

def GA(N=10, gen=10, pop=10, ratio=0.7):
    """
    The main GA code,
    
    Args:
        - N: number of atoms
        - gen: number of generations
        - pop: number of populations
        - ratio: ratio of crossover
    """
    for i in range(gen):
        t0 = time()
        if i == 0:
            cluster = init_population(pop, N)
            
        cluster, fitness = local_optimize(cluster)
        #print(fitness)
        print('best fitness in generation ', i, ':  ', min(fitness), ' time:', time()-t0)
        
        #rank = ranking(fitness)
        #selecton process
        new_cluster = []
        for j in range(pop):
            if j < int(ratio*pop):
                new_cluster.append(crossover(cluster, fitness))
            else:
                new_cluster.append(mutation(cluster, fitness))
        cluster = new_cluster
        
#LJ13: -44.326801
#LJ30: -128.286571

In [2]:
GA(13, 10, 10) 

best fitness in generation  0 :   -39.69967312753056  time: 7.002310037612915
best fitness in generation  1 :   -40.26776965313317  time: 4.991734981536865
best fitness in generation  2 :   -41.390650916223905  time: 4.789714097976685
best fitness in generation  3 :   -39.529817334460105  time: 4.7848780155181885
best fitness in generation  4 :   -40.5951853301321  time: 4.587069034576416
best fitness in generation  5 :   -41.44318838437253  time: 4.528568744659424
best fitness in generation  6 :   -41.47102798937094  time: 4.581162929534912
best fitness in generation  7 :   -41.4710071563527  time: 4.735446214675903
best fitness in generation  8 :   -44.32445212441919  time: 4.643539905548096
best fitness in generation  9 :   -41.44416167041823  time: 4.78840708732605


In [3]:
GA(13, 10, 20)

best fitness in generation  0 :   -39.47361072028621  time: 10.063167095184326
best fitness in generation  1 :   -40.69852936844493  time: 10.100061893463135
best fitness in generation  2 :   -39.81281506507173  time: 9.762456178665161
best fitness in generation  3 :   -39.920035844010634  time: 9.974546909332275
best fitness in generation  4 :   -40.95689011048416  time: 9.379140853881836
best fitness in generation  5 :   -39.38399091228575  time: 9.856512069702148
best fitness in generation  6 :   -39.812267294363195  time: 9.635923862457275
best fitness in generation  7 :   -39.68762627752801  time: 9.727127075195312
best fitness in generation  8 :   -40.39915478626404  time: 10.0072340965271
best fitness in generation  9 :   -40.668688515101756  time: 9.571397066116333


In [4]:
GA(13, 20, 20)

best fitness in generation  0 :   -39.58591054304635  time: 10.050824165344238
best fitness in generation  1 :   -44.32553713961365  time: 9.802794218063354
best fitness in generation  2 :   -44.32575454690882  time: 9.571438074111938
best fitness in generation  3 :   -41.47116494632379  time: 9.26037311553955
best fitness in generation  4 :   -41.46767438247953  time: 9.76556921005249
best fitness in generation  5 :   -41.46730391657661  time: 9.932004928588867
best fitness in generation  6 :   -41.47074735794931  time: 9.8695809841156
best fitness in generation  7 :   -41.467330892669096  time: 9.995826959609985
best fitness in generation  8 :   -41.46733611576265  time: 9.856336116790771
best fitness in generation  9 :   -41.4673359619746  time: 9.976699113845825
best fitness in generation  10 :   -41.46733595695825  time: 9.705574989318848
best fitness in generation  11 :   -41.46733595635755  time: 10.013175964355469
best fitness in generation  12 :   -41.46733595601474  time: 9.6

In [5]:
GA(13, 20, 20, 0.1)

best fitness in generation  0 :   -40.04384405176163  time: 9.717046022415161
best fitness in generation  1 :   -38.632114412906965  time: 9.929518699645996
best fitness in generation  2 :   -38.78319490135087  time: 10.061021089553833
best fitness in generation  3 :   -39.36865309045847  time: 10.007993698120117
best fitness in generation  4 :   -41.399614482062574  time: 9.704454183578491
best fitness in generation  5 :   -40.64054200910862  time: 9.983294010162354
best fitness in generation  6 :   -41.10339343167147  time: 9.550175905227661
best fitness in generation  7 :   -43.20779962446972  time: 9.716034889221191
best fitness in generation  8 :   -41.439654140895314  time: 9.833055019378662
best fitness in generation  9 :   -44.32237655379409  time: 9.57637882232666
best fitness in generation  10 :   -41.39258299474287  time: 10.00723910331726
best fitness in generation  11 :   -41.46981395407201  time: 10.034940004348755
best fitness in generation  12 :   -40.60299159352281  ti

## 20.3 Homework

Check the following documentation and use the differential evolution method to perform optimization on LJ clusters
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html#scipy.optimize.differential_evolution